结论：有影响。列的顺序必然不能乱。

In [1]:
import xgboost as xgb
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
import pandas as pd
import numpy as np
import tqdm, datetime, pickle
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

import ngboost as ngb

from sklearn import datasets

import plotnine
from plotnine import *

import seaborn as sns

# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Activation, Flatten

In [2]:
data_train = pd.read_csv("preprocessedDataset/pre1.csv")
numerical_category_fewValues = [
    "homeOwnership", 
    "verificationStatus",
    "initialListStatus",
    "applicationType",
    "n11",
    "n12",
]
numerical_category_manyValues = [
    "regionCode",
    "employmentTitle",
    "purpose",
    "postCode",
    "title",
]
date_type = [
    'issueDate',
    'earliesCreditLine',
    'issueDateDT',
    'earliesCreditLineDT',
    'earliesCreditLineYear',
    'earliesCreditLineMonth',
    'issueYear',
    'issueMonth'
]
numerical_serial = [
    "loanAmnt","interestRate","installment","annualIncome","dti","delinquency_2years","ficoRangeLow","ficoRangeHigh","openAcc",
    "pubRec","pubRecBankruptcies","revolBal","revolUtil","totalAcc","n0","n1","n2","n3",
    "n4","n5","n6","n7","n8","n9","n10","n13","n14",
    "term", 
]
object_serial = [
    "grade",
    "subGrade", 
    "employmentLength"
]

In [3]:
features = [f for f in data_train.columns if f not in ['id', 'isDefault', "policyCode", "issueDate", "earliesCreditLine"] and '_outliers' not in f]
y_full = data_train['isDefault']
x_full = data_train[features]

In [4]:
from copy import deepcopy
from random import randint

def shuffle(lst):
    temp_lst = deepcopy(lst)
    m = len(temp_lst)
    while (m):
        m -= 1
        i = randint(0, m)
        temp_lst[m], temp_lst[i] = temp_lst[i], temp_lst[m]
    return temp_lst

features = shuffle(features)

In [5]:
x_full = x_full[features]

In [6]:
from sklearn.model_selection import train_test_split

X_, X_test, y_, y_test = train_test_split(x_full, y_full, test_size=0.5, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_, y_, test_size=0.6, random_state=0)
del X_, y_

In [7]:
xgb_model = xgb.Booster()
xgb_model.load_model("trainedModel/xgb.json")

In [8]:
lgb_model = lgb.Booster(model_file='trainedModel/lgb.json')

In [9]:
proba_xgb = xgb_model.predict(
    xgb.DMatrix(X_test.to_numpy())
)
proba_xgb[:10]

array([0.11767495, 0.07059092, 0.22284453, 0.255933  , 0.15696532,
       0.1303988 , 0.17525958, 0.08638427, 0.28543937, 0.20266764],
      dtype=float32)

In [10]:
proba_lgb = lgb_model.predict(
    X_test.to_numpy()
)
proba_lgb[:10]

array([0.1051176 , 0.09611536, 0.08662769, 0.11381645, 0.09870612,
       0.11577522, 0.08210272, 0.0795115 , 0.0894371 , 0.10454692])

In [11]:
cross = pd.DataFrame(
    {
        k:v for v, k in zip(
            [proba_xgb, proba_lgb, y_test], 
            ["proba_xgb", "proba_lgb", "label"]
        )
    }
)
cross.head()

,proba_xgb,proba_lgb,label
158171,0.117675,0.105118,0
480078,0.070591,0.096115,0
258275,0.222845,0.086628,1
20237,0.255933,0.113816,0
354111,0.156965,0.098706,0


In [12]:
import sklearn
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

cols_score = ['auc', 'precision', 'precision_lift', 'recall',  'f1_score']
score_comparison = pd.DataFrame(columns=cols_score, index=None)

for colName in ["proba_xgb", "proba_lgb"]: ## the cols are `cross` table
    auc = roc_auc_score(
        y_test, 
        cross.loc[:, colName]
    )
    fpr,tpr,threshes = sklearn.metrics.roc_curve(
        np.array(y_test), 
        cross.loc[:, colName]
    )
    ks = np.argmax(tpr-fpr)
    thresh = threshes[ks]
    
    pred_col_name = f"{colName}-pred"
    cross[pred_col_name] = -1
    cross.loc[cross[colName] > thresh, pred_col_name] = 1
    cross.loc[cross[colName] <= thresh, pred_col_name] = 0
    cross[pred_col_name].astype(np.int64)
    
    precision = precision_score(cross["label"], cross[pred_col_name])
    recall = recall_score(cross["label"], cross[pred_col_name])
    f1_score_ = f1_score(cross["label"], cross[pred_col_name])
    pn_ratio = cross["label"].sum()/len(cross["label"])
    precision_lift = precision/pn_ratio
    
    score_comparison = score_comparison.append({'auc': auc, 'precision': precision, 'precision_lift': precision_lift, 'recall': recall, 'f1_score': f1_score_}, ignore_index=True)   

score_comparison.index=["proba_xgb", "proba_lgb"]
score_comparison

,auc,precision,precision_lift,recall,f1_score
proba_xgb,0.493646,0.208251,1.040356,0.082529,0.118211
proba_lgb,0.461252,0.290070,1.449102,0.018534,0.034842
